# Buscanto os Valores das Ações

In [62]:
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

yf.pdr_override()

In [63]:
# Especifique o símbolo da ação que você deseja buscar
portfolio = ["VALE3.SA", "PETR4.SA"]  # Apple Inc. como exemplo

initial_date = "2023-07-01"
final_date = "2023-09-07"

In [64]:
# Obtenha os dados da cotação
prices = pdr.get_data_yahoo(portfolio, initial_date, final_date)

# Exiba a cotação
prices.head()

[*********************100%%**********************]  2 of 2 completed


Adj Close                 Close                  High             \
             PETR4.SA   VALE3.SA   PETR4.SA   VALE3.SA   PETR4.SA   VALE3.SA   
Date                                                                           
2023-07-03  28.975494  64.290443  30.080000  66.230003  30.180000  66.500000   
2023-07-04  29.042923  63.970108  30.150000  65.900002  30.389999  66.459999   
2023-07-05  29.004393  63.407089  30.110001  65.320000  30.549999  65.860001   
2023-07-06  28.561283  62.950855  29.650000  64.849998  30.000000  65.589996   
2023-07-07  28.416790  63.542992  29.500000  65.459999  30.200001  65.599998   

                  Low                  Open               Volume            
             PETR4.SA   VALE3.SA   PETR4.SA   VALE3.SA  PETR4.SA  VALE3.SA  
Date                                                                        
2023-07-03  29.719999  64.639999  29.719999  64.709999  31266900  22738400  
2023-07-04  29.959999  65.800003  29.959999  65.839996  18308000  12573200  
2023-07-05  29.660000  65.010002  30.160000  65.610001  53585500  27363600  
2023-07-06  29.150000  64.440002  29.850000  64.720001  47685700  18579200  
2023-07-07  29.360001  64.309998  29.760000  64.480003  55533500  57287600

In [65]:
prices.columns

MultiIndex([('Adj Close', 'PETR4.SA'),
            ('Adj Close', 'VALE3.SA'),
            (    'Close', 'PETR4.SA'),
            (    'Close', 'VALE3.SA'),
            (     'High', 'PETR4.SA'),
            (     'High', 'VALE3.SA'),
            (      'Low', 'PETR4.SA'),
            (      'Low', 'VALE3.SA'),
            (     'Open', 'PETR4.SA'),
            (     'Open', 'VALE3.SA'),
            (   'Volume', 'PETR4.SA'),
            (   'Volume', 'VALE3.SA')],
           )

In [66]:
def filter_stock(prices, stock):
    return {
    'x': prices.index,
    'open': prices.Open[stock],
    'close': prices.Close[stock],
    'high': prices.High[stock],
    'low': prices.Low[stock],
    'type': 'candlestick',
    'name': stock,
    'showlegend': False
    }

In [67]:
def interactive_chart(prices, stock):

    data = [filter_stock(prices, stock)]
    layout = go.Layout()
    fig = go.Figure(data=data, layout=layout)
    fig.show()

In [68]:
interactive_chart(prices, 'PETR4.SA')